# Rank repartition

This notebook is intended for exploring the data fetched from the riot games api in the [Data collection Notebook](data_collection.ipynb).

This will be my first time using intensively a vizualisation library in python, I will be using [Vega Altair](https://altair-viz.github.io/) since it integrates really well with notebooks from what I heard. Also it is build around pandas Dataframe so I hope to get more knowledge of pandas by working with vega altair.

In [ ]:
import sqlite3
import os
import pandas as pd
import altair as alt


def query_player_count_by_rank(db_path):
    """
    Queries the number of players grouped by rank from the database.
    """

    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database file not found: {db_path}")

    conn = sqlite3.connect(db_path)
    
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT tier, rank, COUNT(*) FROM ranked_players GROUP BY tier, rank ORDER BY tier, rank DESC")
        count = cursor.fetchall()
        
        return count
    finally:
        # Close the database connection
        conn.close()


if __name__ == "__main__":
    # Example usage
    db_path = "ranked_players_copy.db"  # Replace with your database path
    try:
        players_by_rank = query_player_count_by_rank(db_path)
        data = pd.DataFrame({
            'tier_rank': [f"{x[0]} {x[1]}" for x in players_by_rank],
            'count': [x[2] for x in players_by_rank]
        })
        chart = alt.Chart(data).mark_bar().encode(
            x='tier_rank',
            y='count',
        )

        
    except Exception as e:
        print(f"An error occurred: {e}")
